In [7]:
from vllm import LLM
import torch

In [8]:
loaded_directions = torch.load('directions.pth')


In [10]:
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1", enable_cv=True)

INFO 06-24 02:31:37 llm_engine.py:166] Initializing an LLM engine (v0.5.0.post1) with config: model='mistralai/Mistral-7B-Instruct-v0.1', speculative_config=None, tokenizer='mistralai/Mistral-7B-Instruct-v0.1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=mistralai/Mistral-7B-Instruct-v0.1)
INFO 06-24 02:31:38 selector.py:161] Cannot use FlashAttention-2 backend due to sliding window.
INFO 06-24 02:31:38 selector.py:52] Using XFormers backend.
INFO 06-24 02:31:44 selector

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

INFO 06-24 02:34:32 model_runner.py:171] Loading model weights took 13.4966 GB
HERE
INFO 06-24 02:34:37 gpu_executor.py:85] # GPU blocks: 11432, # CPU blocks: 2048
INFO 06-24 02:34:40 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-24 02:34:40 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-24 02:34:50 model_runner.py:1013] Graph capturing finished in 10 secs.


In [11]:
from vllm.control_vectors.request import ControlVectorRequest
from vllm.control_vectors.control import ControlVector


cv = ControlVector(model_type="mistral", directions=loaded_directions)

In [12]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0,
    max_tokens=100,
    stop=["[/assistant]"]
)

In [13]:
prompts = [
    "what are apples",
    "what are apples"
]

In [14]:
outputs = llm.generate(
    prompts,
    sampling_params,
    control_vector_request=ControlVectorRequest("high adapter", cv, coefficient=0.01)
)



Processed prompts: 100%|██████████| 2/2 [00:13<00:00,  6.50s/it, est. speed input: 0.77 toks/s, output: 13.15 toks/s]


In [15]:
outputs

[RequestOutput(request_id=0, prompt='what are apples', prompt_token_ids=[1, 767, 460, 979, 2815], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='?\nA: A type of fruit that is round, usually red or green, and has a firm, white flesh. Apples are grown on trees in orchards and are often eaten fresh or used in cooking, baking, and juicing. They are a popular source of fiber, vitamin C, and antioxidants.', token_ids=[28804, 13, 28741, 28747, 330, 1212, 302, 10607, 369, 349, 3713, 28725, 4312, 2760, 442, 5344, 28725, 304, 659, 264, 5799, 28725, 3075, 14063, 28723, 4002, 2815, 460, 10039, 356, 7099, 297, 442, 338, 2178, 304, 460, 2608, 21718, 6138, 442, 1307, 297, 13198, 28725, 22411, 28725, 304, 5666, 9065, 28723, 1306, 460, 264, 4387, 2832, 302, 18867, 28725, 11781, 8255, 334, 28725, 304, 2725, 691, 28744, 313, 1549, 28723, 2], cumulative_logprob=-20.925995858664578, logprobs=None, finish_reason=stop, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=171